# Adv Games Regression
<a id="top"></a>

Treating games as the sum of each team's players stats.
0. [Helper Methods](#helpers)
1. [Score Differential Regressor](#score-diff)
    * [SVD regression](#score-diff-lr), [RFT regression](#score-diff-logr)
    * Only take raw game scores and betting lines
2. [Past Win% Differential Regressor](#winper)
    * [SVD regression](#winper-lr), [RFT regression](#winper-logr)
    * Take past win% over 10, 25, 50, 75, 100 games for each team
3. [Box Score Regressor](#boxscore)
    * Takes in overall box score for either team 

In [59]:
import pandas as pd
import numpy as np
import plotly.express as px

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.decomposition import PCA

In [96]:
pd.set_option('display.max_columns', 250)

## Helper Methods
[to top](#top)
<a id="helpers">

In [71]:
def SVGRegression(X_input, y_input):
    n_folds = 5
    kf = KFold(n_folds, shuffle=True)

    # Create a pipeline that first standardizes the data, then applies PCA, and finally trains the model
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components="mle")),  # k is the number of principal components
        ('regressor', SVR())
    ])

    # Perform cross-validation
    cv_scores = cross_val_score(pipeline, X_input, np.ravel(y_input), cv=kf, scoring='neg_mean_squared_error')

    # Calculate average MSE
    average_mse = -np.mean(cv_scores)
    print("cv:", cv_scores)
    print("Average MSE:", average_mse)
    print("Average RMSE:", np.sqrt(average_mse))

    return pipeline

    # X_train, X_test, y_train, y_test = train_test_split(X_input, y_input, test_size=.1)

    # print(f"X: {X_input.columns.tolist()}")
    # print(f"y: {y_input.columns.tolist()}")

    # # Standardize the data
    # scaler = StandardScaler()
    # X_train_scaled = scaler.fit_transform(X_train)
    # X_test_scaled = scaler.transform(X_test)

    # # Apply PCA
    # pca = PCA(n_components="mle")  # k is the number of principal components you want to keep
    # X_train_pca = pca.fit_transform(X_train_scaled)
    # X_test_pca = pca.transform(X_test_scaled)
    
    # model = SVR()
    # model.fit(X_train_pca, np.ravel(y_train))
    # y_pred = model.predict(X_test_pca)

    # # The coefficients
    # print("Coefficients: \n", model.dual_coef_)
    # # The mean squared error
    # print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
    # # The coefficient of determination: 1 is perfect prediction
    # print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))
    # print()
    # return 

## Rolling Regressors
[to top](#top)

-  [linear regression](#winper-lr)
-  [logistic regression](#winper-logr)
<a id="winper">

In [61]:
df = pd.read_csv("data_proc/games_rolling_master.csv")
df.dropna(inplace=True)
# display(df)
game_info = ['TEAM_ID_T1', 'TEAM_ID_T2', 'DATE', 'away_prob', 'away_spread', 'away_total'] 
intervals = ['10g_', '25g_', '50g_', '75g_', '100g']
columns = game_info + [c for c in df.columns.to_list() if c not in game_info and c[:4] in intervals]
print(columns)

['TEAM_ID_T1', 'TEAM_ID_T2', 'DATE', 'away_prob', 'away_spread', 'away_total', '10g_FGM_T1', '10g_FGM_T2', '10g_FGA_T1', '10g_FGA_T2', '10g_FG3M_T1', '10g_FG3M_T2', '10g_FG3A_T1', '10g_FG3A_T2', '10g_FTM_T1', '10g_FTM_T2', '10g_FTA_T1', '10g_FTA_T2', '10g_OREB_T1', '10g_OREB_T2', '10g_DREB_T1', '10g_DREB_T2', '10g_REB_T1', '10g_REB_T2', '10g_AST_T1', '10g_AST_T2', '10g_STL_T1', '10g_STL_T2', '10g_BLK_T1', '10g_BLK_T2', '10g_TO_T1', '10g_TO_T2', '10g_PF_T1', '10g_PF_T2', '10g_PTS_T1', '10g_PTS_T2', '10g_PLUS_MINUS_T1', '10g_PLUS_MINUS_T2', '10g_FG_PCT_T1', '10g_FG_PCT_T2', '10g_FG3_PCT_T1', '10g_FG3_PCT_T2', '10g_FT_PCT_T1', '10g_FT_PCT_T2', '10g_SCORE_DIFF_T1', '10g_SCORE_DIFF_T2', '10g_WON_T1', '10g_WON_T2', '25g_FGM_T1', '25g_FGM_T2', '25g_FGA_T1', '25g_FGA_T2', '25g_FG3M_T1', '25g_FG3M_T2', '25g_FG3A_T1', '25g_FG3A_T2', '25g_FTM_T1', '25g_FTM_T2', '25g_FTA_T1', '25g_FTA_T2', '25g_OREB_T1', '25g_OREB_T2', '25g_DREB_T1', '25g_DREB_T2', '25g_REB_T1', '25g_REB_T2', '25g_AST_T1', '25g_AS

In [72]:
reg = SVGRegression(df[columns], df[["SCORE_DIFF_T2"]])
                
# SVGRegression(df[['TEAM1_ID', 'TEAM2_ID', 'DATE', 'TEAM1_10wp', 'TEAM1_25wp', 'TEAM1_50wp', 'TEAM1_75wp', 'TEAM1_100wp', 'TEAM2_10wp', 'TEAM2_25wp', 'TEAM2_50wp', 'TEAM2_75wp', 'TEAM2_100wp']],df[["SCORE_DIFF_T1"]])

# SVGRegression(df.drop(columns=['SCORE_DIFF_T1', 'SCORE_DIFF_T2', '']), df[["SCORE_DIFF_T1"]])
# fig = px.scatter(df, x="away_spread", y="SCORE_DIFF", hover_data=["SBR_GAME_ID"], trendline="ols")
# fig.show()


cv: [-176.72584056 -171.77180628 -184.56891564 -165.55873189 -176.43481697]
Average MSE: 175.01202226811034
Average RMSE: 13.229210946542139


In [101]:
def testIndex(i):
    display(df.iloc[:-i][columns])
    display(df.iloc[:-i]['SCORE_DIFF_T2'])
    display(df.iloc[-i]['SCORE_DIFF_T2'])
    display(df.iloc[-i].to_frame().T)


    reg.fit(df.iloc[:-i][columns], df.iloc[:-i]['SCORE_DIFF_T2'])
    display(f"predict: {reg.predict([df.iloc[-i][columns]])}")

testIndex(275)

,TEAM_ID_T1,TEAM_ID_T2,DATE,away_prob,away_spread,away_total,10g_FGM_T1,10g_FGM_T2,10g_FGA_T1,10g_FGA_T2,10g_FG3M_T1,10g_FG3M_T2,10g_FG3A_T1,10g_FG3A_T2,10g_FTM_T1,10g_FTM_T2,10g_FTA_T1,10g_FTA_T2,10g_OREB_T1,10g_OREB_T2,10g_DREB_T1,10g_DREB_T2,10g_REB_T1,10g_REB_T2,10g_AST_T1,10g_AST_T2,10g_STL_T1,10g_STL_T2,10g_BLK_T1,10g_BLK_T2,10g_TO_T1,10g_TO_T2,10g_PF_T1,10g_PF_T2,10g_PTS_T1,10g_PTS_T2,10g_PLUS_MINUS_T1,10g_PLUS_MINUS_T2,10g_FG_PCT_T1,10g_FG_PCT_T2,10g_FG3_PCT_T1,10g_FG3_PCT_T2,10g_FT_PCT_T1,10g_FT_PCT_T2,10g_SCORE_DIFF_T1,10g_SCORE_DIFF_T2,10g_WON_T1,10g_WON_T2,25g_FGM_T1,25g_FGM_T2,25g_FGA_T1,25g_FGA_T2,25g_FG3M_T1,25g_FG3M_T2,25g_FG3A_T1,25g_FG3A_T2,25g_FTM_T1,25g_FTM_T2,25g_FTA_T1,25g_FTA_T2,25g_OREB_T1,25g_OREB_T2,25g_DREB_T1,25g_DREB_T2,25g_REB_T1,25g_REB_T2,25g_AST_T1,25g_AST_T2,25g_STL_T1,25g_STL_T2,25g_BLK_T1,25g_BLK_T2,25g_TO_T1,25g_TO_T2,25g_PF_T1,25g_PF_T2,25g_PTS_T1,25g_PTS_T2,25g_PLUS_MINUS_T1,25g_PLUS_MINUS_T2,25g_FG_PCT_T1,25g_FG_PCT_T2,25g_FG3_PCT_T1,25g_FG3_PCT_T2,25g_FT_PCT_T1,25g_FT_PCT_T2,25g_SCORE_DIFF_T1,25g_SCORE_DIFF_T2,25g_WON_T1,25g_WON_T2,50g_FGM_T1,50g_FGM_T2,50g_FGA_T1,50g_FGA_T2,50g_FG3M_T1,50g_FG3M_T2,50g_FG3A_T1,50g_FG3A_T2,50g_FTM_T1,50g_FTM_T2,50g_FTA_T1,50g_FTA_T2,50g_OREB_T1,50g_OREB_T2,50g_DREB_T1,50g_DREB_T2,50g_REB_T1,50g_REB_T2,50g_AST_T1,50g_AST_T2,50g_STL_T1,50g_STL_T2,50g_BLK_T1,50g_BLK_T2,50g_TO_T1,50g_TO_T2,50g_PF_T1,50g_PF_T2,50g_PTS_T1,50g_PTS_T2,50g_PLUS_MINUS_T1,50g_PLUS_MINUS_T2,50g_FG_PCT_T1,50g_FG_PCT_T2,50g_FG3_PCT_T1,50g_FG3_PCT_T2,50g_FT_PCT_T1,50g_FT_PCT_T2,50g_SCORE_DIFF_T1,50g_SCORE_DIFF_T2,50g_WON_T1,50g_WON_T2,75g_FGM_T1,75g_FGM_T2,75g_FGA_T1,75g_FGA_T2,75g_FG3M_T1,75g_FG3M_T2,75g_FG3A_T1,75g_FG3A_T2,75g_FTM_T1,75g_FTM_T2,75g_FTA_T1,75g_FTA_T2,75g_OREB_T1,75g_OREB_T2,75g_DREB_T1,75g_DREB_T2,75g_REB_T1,75g_REB_T2,75g_AST_T1,75g_AST_T2,75g_STL_T1,75g_STL_T2,75g_BLK_T1,75g_BLK_T2,75g_TO_T1,75g_TO_T2,75g_PF_T1,75g_PF_T2,75g_PTS_T1,75g_PTS_T2,75g_PLUS_MINUS_T1,75g_PLUS_MINUS_T2,75g_FG_PCT_T1,75g_FG_PCT_T2,75g_FG3_PCT_T1,75g_FG3_PCT_T2,75g_FT_PCT_T1,75g_FT_PCT_T2,75g_SCORE_DIFF_T1,75g_SCORE_DIFF_T2,75g_WON_T1,75g_WON_T2,100g_FGM_T1,100g_FGM_T2,100g_FGA_T1,100g_FGA_T2,100g_FG3M_T1,100g_FG3M_T2,100g_FG3A_T1,100g_FG3A_T2,100g_FTM_T1,100g_FTM_T2,100g_FTA_T1,100g_FTA_T2,100g_OREB_T1,100g_OREB_T2,100g_DREB_T1,100g_DREB_T2,100g_REB_T1,100g_REB_T2,100g_AST_T1,100g_AST_T2,100g_STL_T1,100g_STL_T2,100g_BLK_T1,100g_BLK_T2,100g_TO_T1,100g_TO_T2,100g_PF_T1,100g_PF_T2,100g_PTS_T1,100g_PTS_T2,100g_PLUS_MINUS_T1,100g_PLUS_MINUS_T2,100g_FG_PCT_T1,100g_FG_PCT_T2,100g_FG3_PCT_T1,100g_FG3_PCT_T2,100g_FT_PCT_T1,100g_FT_PCT_T2,100g_SCORE_DIFF_T1,100g_SCORE_DIFF_T2,100g_WON_T1,100g_WON_T2
1401,1.610613e+09,1.610613e+09,20160301.0,0.338983,5.500000,214.000000,40.4,43.2,88.6,88.7,10.8,13.2,29.9,33.3,12.9,17.5,17.1,23.4,9.6,9.2,36.5,36.0,46.1,45.2,27.3,28.7,9.8,8.7,6.5,4.7,15.5,15.1,18.2,21.4,104.5,117.1,31.0,33.0,0.457888,0.489153,0.377693,0.401488,0.749500,0.748964,6.2,6.6,0.5,0.9,40.28,42.60,86.84,87.16,10.20,12.72,29.24,30.44,14.08,16.76,18.16,22.04,8.80,9.12,34.40,36.48,43.20,45.60,27.04,29.36,9.96,7.84,6.08,5.12,15.20,14.52,20.04,20.72,104.84,114.68,27.0,41.6,0.464365,0.489818,0.356173,0.415031,0.767829,0.759583,5.40,8.32,0.52,0.88,39.32,42.40,84.64,86.66,9.66,12.90,27.66,30.48,15.44,17.54,19.42,23.18,8.44,9.94,33.08,36.00,41.52,45.94,26.36,29.06,9.72,8.32,5.80,5.88,14.64,15.12,19.24,20.54,103.74,115.24,20.6,53.4,0.465566,0.490592,0.353307,0.422331,0.789211,0.758526,4.12,10.68,0.60,0.92,38.893333,41.533333,84.386667,86.973333,9.693333,12.373333,27.573333,30.266667,15.640000,16.973333,19.813333,22.906667,8.706667,10.746667,32.506667,35.640000,41.213333,46.386667,26.253333,27.973333,9.706667,8.160000,5.226667,5.840000,14.013333,14.693333,18.866667,21.506667,103.120000,112.413333,17.200000,49.400000,0.462062,0.478923,0.353909,0.408793,0.786869,0.744027,3.440000,9.880000,0.586667,0.880000,38.61,41.42,83.93,87.13,9.72,12.00,27.25,29.49,15.80,16.48,19.91,22.00,8.70,10.68,32.11,35.17,40.81,45.8

1401     4.0
1430   -10.0
1443    -3.0
1483    12.0
1485   -24.0
        ... 
9367     9.0
9368    13.0
9369   -25.0
9370    21.0
9371    -8.0
Name: SCORE_DIFF_T2, Length: 7832, dtype: float64

-10.0

,GAME_ID,SBR_GAME_ID,DATE,TEAM_ID_T1,FGM_T1,FGA_T1,FG3M_T1,FG3A_T1,FTM_T1,FTA_T1,OREB_T1,DREB_T1,REB_T1,AST_T1,STL_T1,BLK_T1,TO_T1,PF_T1,PTS_T1,PLUS_MINUS_T1,FG_PCT_T1,FG3_PCT_T1,FT_PCT_T1,SCORE_DIFF_T1,WON_T1,TEAM_ID_T2,FGM_T2,FGA_T2,FG3M_T2,FG3A_T2,FTM_T2,FTA_T2,OREB_T2,DREB_T2,REB_T2,AST_T2,STL_T2,BLK_T2,TO_T2,PF_T2,PTS_T2,PLUS_MINUS_T2,FG_PCT_T2,FG3_PCT_T2,FT_PCT_T2,SCORE_DIFF_T2,WON_T2,away_prob,away_spread,away_total,10g_FGM_T1,10g_FGM_T2,10g_FGA_T1,10g_FGA_T2,10g_FG3M_T1,10g_FG3M_T2,10g_FG3A_T1,10g_FG3A_T2,10g_FTM_T1,10g_FTM_T2,10g_FTA_T1,10g_FTA_T2,10g_OREB_T1,10g_OREB_T2,10g_DREB_T1,10g_DREB_T2,10g_REB_T1,10g_REB_T2,10g_AST_T1,10g_AST_T2,10g_STL_T1,10g_STL_T2,10g_BLK_T1,10g_BLK_T2,10g_TO_T1,10g_TO_T2,10g_PF_T1,10g_PF_T2,10g_PTS_T1,10g_PTS_T2,10g_PLUS_MINUS_T1,10g_PLUS_MINUS_T2,10g_FG_PCT_T1,10g_FG_PCT_T2,10g_FG3_PCT_T1,10g_FG3_PCT_T2,10g_FT_PCT_T1,10g_FT_PCT_T2,10g_SCORE_DIFF_T1,10g_SCORE_DIFF_T2,10g_WON_T1,10g_WON_T2,25g_FGM_T1,25g_FGM_T2,25g_FGA_T1,25g_FGA_T2,25g_FG3M_T1,25g_FG3M_T2,25g_FG3A_T1,25g_FG3A_T2,25g_FTM_T1,25g_FTM_T2,25g_FTA_T1,25g_FTA_T2,25g_OREB_T1,25g_OREB_T2,25g_DREB_T1,25g_DREB_T2,25g_REB_T1,25g_REB_T2,25g_AST_T1,25g_AST_T2,25g_STL_T1,25g_STL_T2,25g_BLK_T1,25g_BLK_T2,25g_TO_T1,25g_TO_T2,25g_PF_T1,25g_PF_T2,25g_PTS_T1,25g_PTS_T2,25g_PLUS_MINUS_T1,25g_PLUS_MINUS_T2,25g_FG_PCT_T1,...,50g_FGM_T2,50g_FGA_T1,50g_FGA_T2,50g_FG3M_T1,50g_FG3M_T2,50g_FG3A_T1,50g_FG3A_T2,50g_FTM_T1,50g_FTM_T2,50g_FTA_T1,50g_FTA_T2,50g_OREB_T1,50g_OREB_T2,50g_DREB_T1,50g_DREB_T2,50g_REB_T1,50g_REB_T2,50g_AST_T1,50g_AST_T2,50g_STL_T1,50g_STL_T2,50g_BLK_T1,50g_BLK_T2,50g_TO_T1,50g_TO_T2,50g_PF_T1,50g_PF_T2,50g_PTS_T1,50g_PTS_T2,50g_PLUS_MINUS_T1,50g_PLUS_MINUS_T2,50g_FG_PCT_T1,50g_FG_PCT_T2,50g_FG3_PCT_T1,50g_FG3_PCT_T2,50g_FT_PCT_T1,50g_FT_PCT_T2,50g_SCORE_DIFF_T1,50g_SCORE_DIFF_T2,50g_WON_T1,50g_WON_T2,75g_FGM_T1,75g_FGM_T2,75g_FGA_T1,75g_FGA_T2,75g_FG3M_T1,75g_FG3M_T2,75g_FG3A_T1,75g_FG3A_T2,75g_FTM_T1,75g_FTM_T2,75g_FTA_T1,75g_FTA_T2,75g_OREB_T1,75g_OREB_T2,75g_DREB_T1,75g_DREB_T2,75g_REB_T1,75g_REB_T2,75g_AST_T1,75g_AST_T2,75g_STL_T1,75g_STL_T2,75g_BLK_T1,75g_BLK_T2,75g_TO_T1,75g_TO_T2,75g_PF_T1,75g_PF_T2,75g_PTS_T1,75g_PTS_T2,75g_PLUS_MINUS_T1,75g_PLUS_MINUS_T2,75g_FG_PCT_T1,75g_FG_PCT_T2,75g_FG3_PCT_T1,75g_FG3_PCT_T2,75g_FT_PCT_T1,75g_FT_PCT_T2,75g_SCORE_DIFF_T1,75g_SCORE_DIFF_T2,75g_WON_T1,75g_WON_T2,100g_FGM_T1,100g_FGM_T2,100g_FGA_T1,100g_FGA_T2,100g_FG3M_T1,100g_FG3M_T2,100g_FG3A_T1,100g_FG3A_T2,100g_FTM_T1,100g_FTM_T2,100g_FTA_T1,100g_FTA_T2,100g_OREB_T1,100g_OREB_T2,100g_DREB_T1,100g_DREB_T2,100g_REB_T1,100g_REB_T2,100g_AST_T1,100g_AST_T2,100g_STL_T1,100g_STL_T2,100g_BLK_T1,100g_BLK_T2,100g_TO_T1,100g_TO_T2,100g_PF_T1,100g_PF_T2,100g_PTS_T1,100g_PTS_T2,100g_PLUS_MINUS_T1,100g_PLUS_MINUS_T2,100g_FG_PCT_T1,100g_FG_PCT_T2,100g_FG3_PCT_T1,100g_FG3_PCT_T2,100g_FT_PCT_T1,100g_FT_PCT_T2,100g_SCORE_DIFF_T1,100g_SCORE_DIFF_T2,100g_WON_T1,100g_WON_T2
9372,22201172.0,20230402_nba_Phoenix_Oklahoma City,20230402.0,1610612756.0,46.0,81.0,13.0,34.0,23.0,28.0,5.0,33.0,38.0,31.0,4.0,6.0,13.0,30.0,128.0,50.0,0.567901,0.382353,0.821429,10.0,1.0,1610612760.0,36.0,88.0,11.0,32.0,35.0,41.0,13.0,28.0,41.0,21.0,7.0,0.0,8.0,23.0,118.0,-50.0,0.409091,0.34375,0.853659,-10.0,0.0,0.679143,-5.0,237.166667,43.1,41.9,92.9,92.0,10.3,10.9,29.0,32.8,17.4,20.2,21.7,23.9,12.4,10.9,31.4,32.1,43.8,43.0,27.8,21.2,6.8,7.7,6.5,3.0,11.9,11.3,23.2,20.4,113.9,114.9,2.5,-12.5,0.464606,0.455763,0.346852,0.332572,0.811011,0.857071,0.5,-2.5,0.5,0.5,42.8,42.28,90.16,91.92,10.72,11.84,30.72,34.64,17.76,20.88,22.44,25.04,11.8,11.0,33.04,31.56,44.84,42.56,27.72,23.64,7.96,8.48,5.44,2.8,12.16,11.36,22.12,20.28,114.08,117.28,15.8,6.0,0.476078,...,43.12,89.2,91.96,11.7,12.46,31.44,34.18,17.6,19.58,22.32,24.06,11.88,11.0,32.8,31.98,44.68,42.98,27.02,24.36,7.32,8.46,5.06,3.64,12.94,11.84,21.18,20.82,111.7,118.28,3.0,13.2,0.462727,0.469338,0.370861,0.367235,0.795764,0.808796,0.6,2.64,0.52,0.54,42.0,43.253333,89.893333,92.386667,12.24,12.066667,32.52,33.653333,17.12,18.986667,

/Users/isaac/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


'predict: [-0.17018933]'